# 1. Data Loading and Exploration (Unwrapped Script)

This script demonstrates how to load and explore the multi-modal data from the Dareplane project.
It follows the same logic as `1_data_loading_and_exploration.ipynb` but **unwraps all functions**
into a sequential, linear script where each logical block is a cell.
This allows for step-by-step execution and inspection of intermediate variables.

In [1]:
from pathlib import Path
import numpy as np
import pandas as pd
from scipy.io import loadmat
import plotly.express as px
import plotly.graph_objects as go
import os

In [2]:
DATA_ROOT = Path.cwd().parent.parent.joinpath("data")

In [3]:
# This map is used to rename the raw channel names from the .mat file
# to more human-readable names like 'LFP_1', 'ECOG_1', etc.
CHANNEL_MAP = {
    "CECOG_HF_1___01___Array_1___01": "LFP_1",
    "CECOG_HF_1___02___Array_1___02": "LFP_2",
    "CECOG_HF_1___03___Array_1___03": "LFP_3",
    "CECOG_HF_1___04___Array_1___04": "LFP_4",
    "CECOG_HF_1___05___Array_1___05": "LFP_5",
    "CECOG_HF_1___06___Array_1___06": "LFP_6",
    "CECOG_HF_1___07___Array_1___07": "LFP_7",
    "CECOG_HF_1___08___Array_1___08": "LFP_8",
    "CECOG_HF_1___09___Array_2___09": "LFP_9",
    "CECOG_HF_1___10___Array_2___10": "LFP_10",
    "CECOG_HF_1___11___Array_2___11": "LFP_11",
    "CECOG_HF_1___12___Array_2___12": "LFP_12",
    "CECOG_HF_1___13___Array_2___13": "LFP_13",
    "CECOG_HF_1___14___Array_2___14": "LFP_14",
    "CECOG_HF_1___15___Array_2___15": "LFP_15",
    "CECOG_HF_1___16___Array_2___16": "LFP_16",
    "CECOG_HF_2___01___Array_3___01": "ECOG_1",
    "CECOG_HF_2___02___Array_3___02": "ECOG_2",
    "CECOG_HF_2___03___Array_3___03": "ECOG_3",
    "CECOG_HF_2___04___Array_3___04": "ECOG_4",
    "CECOG_HF_2___05___Array_3___05": "EOG_1",
    "CECOG_HF_2___06___Array_3___06": "EOG_2",
    "CECOG_HF_2___07___Array_3___07": "EOG_3",
    "CECOG_HF_2___08___Array_3___08": "EOG_4",
}

In [4]:
session_path = DATA_ROOT.joinpath("sub-P001_ses-day2", "lsl")

In [5]:
fname_xdf = list(
    session_path.glob(
        "*block1_copydraw_off.xdf"
    )
)[0]

In [6]:
file_stem = fname_xdf.stem

In [7]:
ao_directory = fname_xdf.parents[1].joinpath("ao")

In [8]:
# find all AO mat files that start with the file stem
mat_files = list(ao_directory.glob(f"{file_stem}*mat"))

In [9]:
mat_file_path = mat_files[0]
print(f"Found .mat file: {mat_file_path}")

Found .mat file: /home/bobby/repos/latent-neural-dynamics-modeling/data/sub-P001_ses-day2/ao/block1_copydraw_off0001.mat


In [10]:
mat_data = loadmat(mat_file_path)

In [11]:
ref_channel_name = "CECOG_HF_2___01___Array_3___01"
ref_channel_data = mat_data[ref_channel_name][0]

In [12]:
mat_data.keys()

dict_keys(['__header__', '__version__', '__globals__', 'CSEG_1___02___Array_1___02_LEVEL_SEG', 'CSEG_1___02___Array_1___02_LEVEL', 'CSEG_1___02___Array_1___02_KHz', 'CSEG_1___02___Array_1___02_KHz_Orig', 'CSEG_1___02___Array_1___02_BitResolution', 'CSEG_1___02___Array_1___02_Gain', 'CSEG_1___02___Array_1___02_TimeBegin', 'CSEG_1___02___Array_1___02_TimeEnd', 'CSEG_1___04___Array_1___04_LEVEL_SEG', 'CSEG_1___04___Array_1___04_LEVEL', 'CSEG_1___04___Array_1___04_KHz', 'CSEG_1___04___Array_1___04_KHz_Orig', 'CSEG_1___04___Array_1___04_BitResolution', 'CSEG_1___04___Array_1___04_Gain', 'CSEG_1___04___Array_1___04_TimeBegin', 'CSEG_1___04___Array_1___04_TimeEnd', 'CSEG_1___05___Array_1___05_LEVEL_SEG', 'CSEG_1___05___Array_1___05_LEVEL', 'CSEG_1___05___Array_1___05_KHz', 'CSEG_1___05___Array_1___05_KHz_Orig', 'CSEG_1___05___Array_1___05_BitResolution', 'CSEG_1___05___Array_1___05_Gain', 'CSEG_1___05___Array_1___05_TimeBegin', 'CSEG_1___05___Array_1___05_TimeEnd', 'CSEG_1___06___Array_1___06

In [13]:

# The sampling rate is stored in a variable like '..._KHz'.
# It's given in KHz, so we multiply by 1000.
sfreq = int(mat_data[f"{ref_channel_name}_KHz"][0][0]) * 1000

In [14]:
print(f"Sampling frequency: {sfreq} Hz")

Sampling frequency: 22000 Hz


In [15]:
# Create a time vector in seconds.
num_samples = len(ref_channel_data)
duration_sec = num_samples / sfreq
time_vector = np.linspace(0, duration_sec, num_samples, endpoint=False)

# Create the initial DataFrame for the AlphaOmega (AO) neural data.
df_ao = pd.DataFrame({"time": time_vector, "src": "AO"})

# Add all the channels defined in CHANNEL_MAP to the DataFrame.
channel_data_dict = {v: mat_data[k][0] for k, v in CHANNEL_MAP.items()}
df_ao = df_ao.assign(**channel_data_dict)

print("Created DataFrame for AO (neural) data.")
df_ao.info()

Created DataFrame for AO (neural) data.
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1208305 entries, 0 to 1208304
Data columns (total 26 columns):
 #   Column  Non-Null Count    Dtype  
---  ------  --------------    -----  
 0   time    1208305 non-null  float64
 1   src     1208305 non-null  object 
 2   LFP_1   1208305 non-null  int16  
 3   LFP_2   1208305 non-null  int16  
 4   LFP_3   1208305 non-null  int16  
 5   LFP_4   1208305 non-null  int16  
 6   LFP_5   1208305 non-null  int16  
 7   LFP_6   1208305 non-null  int16  
 8   LFP_7   1208305 non-null  int16  
 9   LFP_8   1208305 non-null  int16  
 10  LFP_9   1208305 non-null  int16  
 11  LFP_10  1208305 non-null  int16  
 12  LFP_11  1208305 non-null  int16  
 13  LFP_12  1208305 non-null  int16  
 14  LFP_13  1208305 non-null  int16  
 15  LFP_14  1208305 non-null  int16  
 16  LFP_15  1208305 non-null  int16  
 17  LFP_16  1208305 non-null  int16  
 18  ECOG_1  1208305 non-null  int16  
 19  ECOG_2  1208305 non-nul

In [16]:

# The CPORT stream contains hardware markers sent via a parallel port.
# It's a 2xN array: row 0 is timestamps, row 1 is marker values.
cport_data = mat_data["CPORT__1"]
cport_timestamps = cport_data[0]
cport_values = cport_data[1]

# The timestamps need to be converted to sample indices in our main data array.
cport_khz = mat_data["CPORT__1_KHz"][0][0]
t_start_recording = mat_data[f"{ref_channel_name}_TimeBegin"][0][0]

# Convert CPORT timestamps to seconds and align with the recording start time.
cport_time_sec = (cport_timestamps / (int(cport_khz) * 1000)) - int(t_start_recording)

# Convert aligned time in seconds to sample indices.
cport_indices = (cport_time_sec * sfreq).astype(int).flatten()

# Ignore markers with a value of 0 (hardware reset signals).
valid_marker_mask = cport_values != 0
valid_indices = cport_indices[valid_marker_mask]
valid_values = cport_values[valid_marker_mask]

# Add them to a new 'marker' column in the DataFrame.
valid_indices_in_bounds = valid_indices[
    (valid_indices >= 0) & (valid_indices < len(df_ao))
]
valid_values_in_bounds = valid_values[
    (valid_indices >= 0) & (valid_indices < len(df_ao))
]
df_ao["marker"] = np.nan  # Initialize column
df_ao.loc[valid_indices_in_bounds, "marker"] = valid_values_in_bounds

print(f"Added {len(valid_indices_in_bounds)} hardware markers to the 'marker' column.")
print("Marker summary:\n", df_ao["marker"].value_counts())

Added 6 hardware markers to the 'marker' column.
Marker summary:
 marker
34.0     2
10.0     1
110.0    1
11.0     1
111.0    1
Name: count, dtype: int64


In [17]:
import pyxdf
print(f"Loading LSL data streams from: {fname_xdf}")

Loading LSL data streams from: /home/bobby/repos/latent-neural-dynamics-modeling/data/sub-P001_ses-day2/lsl/block1_copydraw_off.xdf


In [18]:

streams, header = pyxdf.load_xdf(fname_xdf, dejitter_timestamps=False)

In [19]:
ao_lsl_stream = next((s for s in streams if "AODataStream" in s["info"]["name"][0]), None)
marker_stream = next((s for s in streams if "CopyDrawParadigmMarkerStream" in s["info"]["name"][0]), None)
stylus_stream = next((s for s in streams if "Mouse" in s["info"]["name"][0]), None)

In [20]:
if ao_lsl_stream is None or marker_stream is None:
    raise ValueError("Could not find AODataStream or MarkerStream in the XDF file.")

In [21]:
# --- Step 3: Create the LSL signal DataFrame (df_lsl) ---
# The LSL signal is also recorded from the ECoG channels. We'll use the first ECoG channel (index 16) for alignment.
lsl_ecog_channel_index = 16 
df_lsl = pd.DataFrame({
    "time": ao_lsl_stream["time_stamps"],
    "data": ao_lsl_stream["time_series"][:, lsl_ecog_channel_index],
    "src": "LSL"
})
# The LSL timestamps are absolute. We make them relative to their own start time for now.
# The alignment logic will calculate the true offset relative to the AO data later.
t0_lsl = df_lsl["time"].iloc[0]
df_lsl["time"] -= t0_lsl
print("Created DataFrame for LSL signal data (df_lsl).")

Created DataFrame for LSL signal data (df_lsl).


In [22]:
# --- Step 4: Create the LSL marker DataFrame (df_markers) ---
df_markers = pd.DataFrame({
    "time": marker_stream["time_stamps"],
    "marker": [m[0] for m in marker_stream["time_series"]]
})
df_markers["time"] -= t0_lsl # Align marker timestamps to the start of the LSL signal stream
print("Created DataFrame for LSL markers (df_markers).")

Created DataFrame for LSL markers (df_markers).


In [23]:
stylus_stream

{'info': defaultdict(list,
             {'name': ['MouseButtons'],
              'type': ['Markers'],
              'channel_count': ['1'],
              'channel_format': ['string'],
              'source_id': [None],
              'nominal_srate': ['0.000000000000000'],
              'version': ['1.100000000000000'],
              'created_at': ['69.78095000000000'],
              'uid': ['fcb940f9-6db8-4f42-9118-1953c4615fac'],
              'session_id': ['default'],
              'hostname': ['LAPTOP-0RCVEO60'],
              'v4address': [None],
              'v4data_port': ['16575'],
              'v4service_port': ['16575'],
              'v6address': [None],
              'v6data_port': ['16575'],
              'v6service_port': ['16575'],
              'desc': [None],
              'stream_id': 7,
              'effective_srate': np.float64(0.44616112458578966),
              'segments': [(0, 12)]}),
 'footer': {'info': defaultdict(list,
              {'first_timestamp': ['61

In [24]:
stylus_stream["time_series"]

[['MouseButtonLeft pressed'],
 ['MouseButtonLeft released'],
 ['MouseButtonLeft pressed'],
 ['MouseButtonLeft released'],
 ['MouseButtonLeft pressed'],
 ['MouseButtonLeft released'],
 ['MouseButtonLeft pressed'],
 ['MouseButtonLeft released'],
 ['MouseButtonLeft pressed'],
 ['MouseButtonLeft pressed'],
 ['MouseButtonLeft released'],
 ['MouseButtonLeft pressed'],
 ['MouseButtonLeft released']]

In [25]:
# --- Step 5: Create the stylus DataFrame (df_stylus) ---
if stylus_stream:
    # The stylus coordinate stream should have 2 columns of data (x, y).
    # The 'MouseButtons' stream, which was likely picked up, has only 1 column of string data.
    # We check the shape of the time_series to ensure we have the correct stream.
    is_coordinate_stream = (
        isinstance(stylus_stream["time_series"], list) and
        len(stylus_stream["time_series"]) > 0 and
        len(stylus_stream["time_series"][0]) == 2 and
        isinstance(stylus_stream["time_series"][0][0], (int, float))
    )

    if is_coordinate_stream:
        df_stylus = pd.DataFrame(stylus_stream["time_series"], columns=["x", "y"])
        df_stylus["time"] = stylus_stream["time_stamps"]
        df_stylus["time"] -= t0_lsl # Align stylus timestamps to the start of the LSL signal stream
        print("Created DataFrame for stylus data (df_stylus).")
    else:
        # This is likely the 'MouseButtons' stream, not the coordinate stream.
        print(f"A stylus-related stream named '{stylus_stream['info']['name'][0]}' was found, but it does not contain (x, y) coordinates.")
        print("This is likely the button-press stream. Skipping creation of df_stylus.")
        df_stylus = None
else:
    df_stylus = None
    print("Stylus stream not found.")

A stylus-related stream named 'MouseButtons' was found, but it does not contain (x, y) coordinates.
This is likely the button-press stream. Skipping creation of df_stylus.


In [26]:
# --- Step 6: Add LSL markers to the LSL signal DataFrame for the fallback alignment method ---
lsl_marker_indices = [np.searchsorted(df_lsl.time, t) for t in df_markers.time]
df_lsl["marker"] = np.nan
# Ensure indices are within bounds of the dataframe
valid_marker_mask = [idx < len(df_lsl) for idx in lsl_marker_indices]
valid_indices = np.array(lsl_marker_indices)[valid_marker_mask]
valid_values = df_markers.marker.values[valid_marker_mask]
df_lsl.loc[valid_indices, "marker"] = valid_values
print("Added LSL markers to df_lsl.")
df_lsl.info()

Added LSL markers to df_lsl.
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 999144 entries, 0 to 999143
Data columns (total 4 columns):
 #   Column  Non-Null Count   Dtype  
---  ------  --------------   -----  
 0   time    999144 non-null  float64
 1   data    999144 non-null  float32
 2   src     999144 non-null  object 
 3   marker  6 non-null       float64
dtypes: float32(1), float64(2), object(1)
memory usage: 26.7+ MB


In [27]:
# This cell unwraps the logic from `align_on_markers` and `align_on_stim_artifact`.
# It first tries to align using hardware markers, and if that fails, it falls
# back to using the large stimulation artifact.

print("\nCell 7: Aligning AO and LSL data streams.")
aligned_df = None
time_offset = None

# --- Method 1: Align on Hardware Markers (unwrapped from `align_on_markers`) ---
try:
    print("Attempting alignment using hardware markers...")
    # Get the timestamps of non-null markers from both sources
    ao_marker_times = df_ao[df_ao.marker.notna()].time
    lsl_marker_times = df_markers[df_markers.marker.notna()].time

    # Calculate the time difference (delta) between consecutive markers
    dt_ao = np.diff(ao_marker_times)
    dt_lsl = np.diff(lsl_marker_times)

    # Find a matching sequence of marker intervals. This is needed because
    # the AO recording might have extra hardware triggers at the start.
    # We look for a sequence in AO deltas that matches the first LSL delta.
    match_found = False
    ao_start_index = -1
    for i in range(min(3, len(dt_ao))):  # Check first 3 AO deltas
        # Allow for a 10% tolerance in timing
        if np.abs(dt_ao[i] - dt_lsl[0]) / dt_lsl[0] < 0.1:
            ao_start_index = i
            match_found = True
            break

    if not match_found:
        raise KeyError("No matching marker time differences found.")

    print(
        f"Found matching marker sequence starting at AO marker index {ao_start_index}."
    )

    # --- Transfer LSL markers to the AO timeline ---
    # Get the time of the first matched AO marker
    t0_ao = ao_marker_times.iloc[ao_start_index]

    # The times of all subsequent LSL markers relative to the first one
    lsl_marker_relative_times = lsl_marker_times - lsl_marker_times.iloc[0]

    # Calculate the absolute timestamps for LSL markers in the AO time domain
    aligned_lsl_marker_times = t0_ao + lsl_marker_relative_times

    # Find the closest sample index in the AO dataframe for each aligned marker time
    aligned_indices = [np.searchsorted(df_ao.time, t) for t in aligned_lsl_marker_times]

    # Create the new 'lsl_marker' column in the AO dataframe
    aligned_df = df_ao.copy()
    aligned_df["lsl_marker"] = np.nan
    aligned_df.loc[aligned_indices, "lsl_marker"] = df_markers[
        df_markers.marker.notna()
    ].marker.values

    # For visualization, calculate the time offset
    time_offset = ao_marker_times.iloc[ao_start_index] - lsl_marker_times.iloc[0]

    print(
        f"Success: Aligned data using hardware markers. Time offset: {time_offset:.4f}s"
    )
except (KeyError, IndexError) as e:
    print(f"Marker alignment failed ({e}). Falling back to stim artifact alignment.")

    threshold = 2000  # uV, for detecting the large stimulation artifact

    # Find the index of the first major peak (the stim artifact) in both streams
    # We start searching after 1s to avoid startup artifacts.
    ix_lsl_first_peak = df_lsl[(df_lsl.data > threshold) & (df_lsl.time > 1)].index[0]
    ix_ao_first_peak = df_ao[(df_ao.ECOG_1 > threshold) & (df_ao.time > 1)].index[0]

    # Calculate the coarse time offset between the two streams
    coarse_time_offset = (
        df_ao.time.iloc[ix_ao_first_peak] - df_lsl.time.iloc[ix_lsl_first_peak]
    )
    print(f"Coarse offset from stim artifact: {coarse_time_offset:.4f}s")

    # --- Fine-tune the offset with a small cross-correlation ---
    # This corrects for small misalignments by matching the signal shape.
    chk_idx_range = 10_000  # Search window in samples (in AO data)
    chk_len = 400  # Length of the signal chunk to match (in LSL data)

    # We'll match the signal shape around the first LSL marker
    idx_lsl_first_marker = df_lsl[df_lsl.marker.notna()].index[0]
    t_lsl_first_marker = df_lsl.time.iloc[idx_lsl_first_marker]

    # Estimate where this marker should be in the AO data using the coarse offset
    ao_test_idx = np.searchsorted(df_ao.time, t_lsl_first_marker + coarse_time_offset)

    # Extract the chunk of LSL data to be used as a template
    dlsl_chk = df_lsl.iloc[idx_lsl_first_marker : idx_lsl_first_marker + chk_len]

    # Extract a larger search window from the AO data
    dao_chk = df_ao.iloc[ao_test_idx - chk_idx_range : ao_test_idx + chk_idx_range]

    # Slide the LSL chunk across the AO window and find the best match
    differences = np.asarray(
        [
            np.abs(
                dao_chk.iloc[i : i + chk_len].ECOG_1.values - dlsl_chk.data.values
            ).mean()
            for i in range(len(dao_chk) - chk_len)
        ]
    )

    # The shift that minimizes the difference
    idx_shift = np.argmin(differences) - chk_idx_range
    fine_tuned_offset = coarse_time_offset + idx_shift / sfreq
    time_offset = fine_tuned_offset  # Store for visualization
    print(f"Fine-tuned offset after cross-correlation: {fine_tuned_offset:.4f}s")

    # --- Transfer LSL markers to the AO timeline using the final offset ---
    dm = df_markers[df_markers.marker.notna()]
    aligned_indices = [
        np.searchsorted(df_ao.time, t + fine_tuned_offset) for t in dm.time
    ]

    aligned_df = df_ao.copy()
    aligned_df["lsl_marker"] = np.nan
    aligned_df.loc[aligned_indices, "lsl_marker"] = dm.marker.values

    print("Success: Aligned data using stimulation artifact.")




Cell 7: Aligning AO and LSL data streams.
Attempting alignment using hardware markers...
Marker alignment failed ('No matching marker time differences found.'). Falling back to stim artifact alignment.


IndexError: index 0 is out of bounds for axis 0 with size 0

In [ ]:

if aligned_df is not None:
    print(f"Alignment complete. Result is a DataFrame with shape {aligned_df.shape}")
    print(
        f"Found {aligned_df.lsl_marker.notna().sum()} LSL markers now aligned to the AO data."
    )
    print("\nValue counts of aligned markers:")
    print(aligned_df.lsl_marker.value_counts())
    print("\nHead of the DataFrame with a marker:")
    print(aligned_df[aligned_df.lsl_marker.notna()].head())
else:
    print("Alignment failed. `aligned_df` is None.")


In [ ]:
# This is a crucial step to visually confirm the alignment worked. We plot
# the LSL signal (shifted by the calculated offset) on top of the AO signal.
# They should overlap almost perfectly.

print("\nCell 9: Visualizing the alignment quality.")

if time_offset is not None and aligned_df is not None:
    # Create a new DataFrame for plotting LSL data with the aligned time
    df_lsl_aligned = df_lsl.copy()
    df_lsl_aligned["time"] += time_offset

    # Concatenate for easier plotting with Plotly Express
    plot_df = pd.concat(
        [
            df_ao[["time", "ECOG_1", "src"]],
            df_lsl_aligned[["time", "data", "src"]].rename(columns={"data": "ECOG_1"}),
        ]
    )

    # Select a small time window to see the alignment clearly.
    # Let's find the first marker to center the plot around.
    first_marker_time = aligned_df[aligned_df.lsl_marker.notna()].time.iloc[0]
    plot_window_start = first_marker_time - 0.5
    plot_window_end = first_marker_time + 0.5

    plot_df_window = plot_df[
        (plot_df["time"] >= plot_window_start) & (plot_df["time"] <= plot_window_end)
    ]

    fig = px.line(
        plot_df_window,
        x="time",
        y="ECOG_1",
        color="src",
        title="Alignment Check: AO vs LSL (shifted)",
        labels={"time": "Time (s)", "ECOG_1": "Amplitude (uV)"},
    )
    fig.show()
else:
    print("Skipping alignment visualization because alignment failed.")


In [ ]:
print("\nCell 10: Visualizing the stylus trace.")

# The stylus data is in the `df_stylus` DataFrame created in Cell 6.
fig = px.line(df_stylus, x="x", y="y", title="Stylus Trace for the Block")

# Screen coordinates often have the origin at the top-left, so we invert the y-axis.
fig.update_yaxes(autorange="reversed")
fig.update_layout(
    xaxis=dict(scaleanchor="y", scaleratio=1)
)  # Ensure aspect ratio is 1:1
fig.show()
